# Simulate and save firing rate for a cell

## 1. Input parameters

In [1]:
# Load cells ID
import numpy as np
from pathlib import Path

layer_idx  = 1     # 0: L1, 1: L2/3, 2: L4, 3: L5, 4:L6
cell_clone = 4      # There are 5 clones, so put numbers 1-5
base_path = '/home/lascon/Documents/241165/AberraEtAl2018/cells/'

cell_number_pop = 10 # Cell number to create the population 

Cell = [
    'L1_NGCDA_bNAC219_',
    'L23_PC_cADpyr229_',  
    'L4_LBC_cACint209_',
    'L5_TTPC2_cADpyr232_',
    'L6_TPC_L4_cADpyr231_'
]



Layer = "_".join(Cell[layer_idx].split("_")[0:1])

cell_path =f'{base_path}{Cell[layer_idx]}{cell_clone}'

cell_Name_prefix = f'{"_".join(Cell[layer_idx].split("_")[2:3])}_{"_".join(Cell[layer_idx].split("_")[0:2])}_'


cell_label = "_".join(Cell[layer_idx].split("_")[1:2])
cell_type = "_".join(Cell[layer_idx].split("_")[2:3])


current_amps = np.loadtxt(f"{cell_path}/current_amps.dat") 
current_amps = current_amps[3]



print(cell_path)
print(Layer)
print(cell_label)
print(cell_type)
print(cell_Name_prefix)
print(current_amps)



from pathlib import Path

hoc_path = Path(cell_path)/ "template.hoc"   # cell_path puede ser str o Path

text = hoc_path.read_text(errors="ignore").splitlines()

# 1) Mostrar líneas donde aparece el string (con número de línea)
hits = [(i+1, line) for i, line in enumerate(text) if cell_Name_prefix in line]
print(f"Found {len(hits)} matches for '{cell_Name_prefix}' in {hoc_path}")
for ln, line in hits[:20]:  # muestra hasta 20
    print(f"{ln}: {line}")

# 2) Extraer el nombre del template si está en 'begintemplate ...' y empieza con el prefijo
templates = []
for line in text:
    s = line.strip()
    if s.startswith("begintemplate"):
        parts = s.split()
        if len(parts) >= 2 and parts[1].startswith(cell_Name_prefix):
            templates.append(parts[1])

print("Templates matching prefix:", templates)
cell_Name = templates[0]
print(cell_Name)


# Config the cell path to compile .mod files
%cd "{cell_path}/mechanisms"
!nrnivmodl 

/home/lascon/Documents/241165/AberraEtAl2018/cells/L23_PC_cADpyr229_4
L23
PC
cADpyr229
cADpyr229_L23_PC_
0.1825334
Found 2 matches for 'cADpyr229_L23_PC_' in /home/lascon/Documents/241165/AberraEtAl2018/cells/L23_PC_cADpyr229_4/template.hoc
35: begintemplate cADpyr229_L23_PC_c292d67a2e
259: endtemplate cADpyr229_L23_PC_c292d67a2e
Templates matching prefix: ['cADpyr229_L23_PC_c292d67a2e']
cADpyr229_L23_PC_c292d67a2e
/home/lascon/Documents/241165/AberraEtAl2018/cells/L23_PC_cADpyr229_4/mechanisms
/home/lascon/Documents/241165/AberraEtAl2018/cells/L23_PC_cADpyr229_4/mechanisms
Mod files: "./CaDynamics_E2.mod" "./Ca_HVA.mod" "./Ca_LVAst.mod" "./Ih.mod" "./Im.mod" "./K_Pst.mod" "./K_Tst.mod" "./Nap_Et2.mod" "./NaTa_t.mod" "./NaTs2_t.mod" "./ProbAMPANMDA_EMS.mod" "./ProbGABAAB_EMS.mod" "./SK_E2.mod" "./SKv3_1.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../Ca_LVAst.mod
 -> NMODL ../CaDynamics_E2.mod
 -> NMODL ../Ca_HVA.mod
Translating Ca_LVAst.mod int

## 2. Cell and stimulus parameters

In [2]:
from neuron import h
from netpyne import specs, sim

h.chdir(cell_path)


#  load hoc
h.load_file('stdrun.hoc')
h.load_file('nrngui.hoc')
h.load_file('import3d.hoc')
h.load_file('morphology.hoc')
h.load_file('biophysics.hoc')
h.load_file('template.hoc')

# --------------------------------------------------------------

# Cell parameters

# --------------------------------------------------------------

def run_step_netpyne(step_amp, simLabel):
    sim.clearAll()  # important in loops

    netParams = specs.NetParams()
    simConfig = specs.SimConfig()

    # 1) Import the cell template -> it creates one rule com 'secs'
    netParams.importCellParams(
        label= cell_label, 
        conds={'cellType': cell_type},          # pop vai bater nessa cond
        fileName='template.hoc',
        cellName= cell_Name,
        cellArgs=[1],
        importSynMechs=False
    )

    # 2) Population  
    netParams.popParams['pop1'] = {
        'cellType': cell_type,
        'numCells': cell_number_pop
    }

    
    # 3) IClamp
    netParams.stimSourceParams['iclamp'] = {
        'type': 'IClamp',
        'del': 100.0,
        'dur': 600.0,
        'amp': float(step_amp)
    }

    netParams.rotateCellsRandomly = [0, 6.2832]

    # 4) Config sim (first create, then discover the correct name of the soma)
    simConfig.duration = 800.0
    simConfig.dt = 0.025
    simConfig.recordStep = 0.025
    simConfig.filename = simLabel
    simConfig.saveJson = True
    simConfig.createNEURONObj = True
    simConfig.cache_efficient = True
    simConfig.checkErrors = False
    simConfig.analysis = {}
    # simConfig.saveLFPCells = True
    # simConfig.recordLFP = [[50, 100, 50], [50, 200, 50]]
    simConfig.recordLFP = [
        [50, 50,  0],
        [20, 40,  0],
        [0,  45, 55],
    ]


    # criate the network (no stimulation yet) just to inspect 
    sim.create(netParams=netParams, simConfig=simConfig)

    cell = sim.net.cells[0]
    secs = list(cell.secs.keys())
    print("Secs:", secs)

    # take soma automatically
    soma_sec = None
    for cand in ['soma_0', 'soma', 'soma[0]']: #try some usual patterns that soma appears
        if cand in cell.secs:
            soma_sec = cand
            break
    if soma_sec is None:
        raise RuntimeError(f"Soma not foubd. Secs available: {secs}")

    # 5) Now define the stim and record using the correct soma name
    netParams.stimTargetParams['iclamp->pop1'] = {
        'source': 'iclamp',
        'conds': {'pop': 'pop1'},
        'sec': soma_sec,
        'loc': 0.5
    }

    simConfig.recordCells = ['all'] 
    simConfig.recordTraces = {
        'V_soma': {'sec': soma_sec, 'loc': 0.5, 'var': 'v'},
        'V_axon_0': {'sec':'axon_0', 'loc':0.5, 'var':'v'},
        'V_axon_1': {'sec':'axon_1', 'loc':0.5, 'var':'v'}, 
        'V_dend_0': {'sec':'dend_0', 'loc':0.5, 'var':'v'},                            
        'V_dend_1': {'sec':'dend_1', 'loc':0.5, 'var':'v'},            
        'V_dend_2': {'sec':'dend_2', 'loc':0.5, 'var':'v'},             
        'V_dend_3': {'sec':'dend_3', 'loc':0.5, 'var':'v'},
    }
    simConfig.recordStim = True
    simConfig.recordTime = True

    simConfig.analysis['plotTraces'] = {
        'include': ['pop1'], 
        'saveFig': False, 
        'overlay': True, 
        'oneFigPer': 'cell'}  # Plot recorded traces for this list of cells

    # simConfig.analysis['plotTraces'] = {'include': [0], 'saveFig': False}  # Plot recorded traces for this list of cells

    simConfig.analysis['plotConn'] = {'saveFig': False}    

    # 6) recreate from zero and run (to insure that stim+recordTraces work)
    sim.clearAll()
    sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

    sim.gatherData()                  			# gather spiking data and cell info from each node
    sim.saveData()                    			# save params, cell info and sim output to file (pickle,mat,txt,etc)#
    # sim.analysis.plotData() 


    #------------------------------------------------------------------------------
    # Saving
    #------------------------------------------------------------------------------
    simConfig.savePickle = False         	## Save pkl file
    simConfig.saveJson = True	           	## Save json file
    simConfig.saveDataInclude = ['simData'] ## , 'netParams', 'simConfig', ,'simData'
    simConfig.backupCfgFile = None
    simConfig.gatherOnlySimData = False
    simConfig.saveCellSecs = True
    simConfig.saveCellConns = True
        
    
# -------------------------------------------------------

# Compute firing rate

# -------------------------------------------------------


import json
import numpy as np
import os

def compute_firing_rate(step_idx, thresh=-20.0):

    base = cell_path
    fname = os.path.join(base, f'step_{step_idx:02d}_data.json')

    with open(fname) as f:
        data = json.load(f)

    t = np.array(data['simData']['t'])
    # v = np.array(list(data['simData']['V_soma'].values())[0])

    # automatically find the soma voltage trace
    Vtrace = None
    for key, val in data['simData'].items():
        if key.lower().startswith('v'):   # it takes V_soma, V, v_soma_0, etc.
            if isinstance(val, dict) and len(val) > 0:
                Vtrace = np.array(next(iter(val.values())))
                break
            elif isinstance(val, list):
                Vtrace = np.array(val)
                break

    if Vtrace is None:
        raise RuntimeError("voltage no found in the trace, verify recordTraces in simConfig.")

    v = Vtrace

    

    # stim window (same as in protocol)
    stim_mask = (t > 100) & (t < 700)
    t_stim = t[stim_mask]
    v_stim = v[stim_mask]

    # detect matches in the limiar (spikes)
    crossings = (v_stim[:-1] < thresh) & (v_stim[1:] >= thresh)
    spike_times = t_stim[1:][crossings]

    # stim duration in seconds
    stim_dur = (700 - 100) / 1000.0  # 0.6 s

    firing_rate = len(spike_times) / stim_dur  # Hz

    return firing_rate, spike_times


NEURON: unable to open font "*helvetica-medium-r-normal*--14*", using "fixed"


	1 
	1 
	1 


## 3. Run simulation per cell

In [ ]:
run_step_netpyne(current_amps, simLabel=f'step_{0:02d}')

Starting to add synapses
Added excitatory synapse 0 originating from cell 27809 of m-type L6_IPC on basal section 7(0.902000) and dep 667.000000
Added excitatory synapse 1 originating from cell 27809 of m-type L6_IPC on basal section 7(0.887000) and dep 661.000000
Added excitatory synapse 2 originating from cell 27809 of m-type L6_IPC on basal section 7(0.900000) and dep 673.000000
Added inhibitory synapse 3 originating from cell 31411 of m-type L1_DAC on apical section 35(0.637000) and dep 608.000000
Added inhibitory synapse 4 originating from cell 31411 of m-type L1_DAC on apical section 35(0.624000) and dep 499.000000
Added inhibitory synapse 5 originating from cell 31411 of m-type L1_DAC on apical section 26(0.409000) and dep 780.000000
Added inhibitory synapse 6 originating from cell 31411 of m-type L1_DAC on apical section 13(0.506000) and dep 986.000000
Added inhibitory synapse 7 originating from cell 31411 of m-type L1_DAC on apical section 8(0.132000) and dep 582.000000
Added 



Creating network of 1 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 10 
  Done; cell creation time = 0.67 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
  Number of stims on node 0: 0 
  Done; cell stims creation time = 0.00 s.
Secs: ['soma_0', 'dend_0', 'dend_1', 'dend_2', 'dend_3', 'dend_4', 'dend_5', 'dend_6', 'dend_7', 'dend_8', 'dend_9', 'dend_10', 'dend_11', 'dend_12', 'dend_13', 'dend_14', 'dend_15', 'apic_0', 'apic_1', 'apic_2', 'apic_3', 'apic_4', 'apic_5', 'apic_6', 'apic_7', 'apic_8', 'apic_9', 'apic_10', 'apic_11', 'apic_12', 'apic_13', 'apic_14', 'apic_15', 'apic_16', 'apic_17', 'apic_18', 'apic_19', 'apic_20', 'apic_21', 'apic_22', 'apic_23', 'apic_24', 'apic_25', 'apic_26', 'apic_27', 'apic_28', 'apic_29', 'apic_30', 'apic_31', 'apic_32', 'apic_33', 'apic_34', 'apic_35', 'apic_36', 'apic_37', 'apic_38', 'apic_39', 'apic_40', 'apic_41', 'apic_42', 'apic_43', 'apic_44', 'apic_45', 'apic_46', 'apic_47', 'apic_48', 'apic_49', 'apic_50', 'apic_51', 'apic_52', 



Creating network of 1 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 10 
  Done; cell creation time = 0.60 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
Adding stims...
  Number of stims on node 0: 10 
  Done; cell stims creation time = 0.02 s.
Recording 70 traces of 7 types on node 0

Running simulation using NEURON for 800.0 ms...
